In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import json
import os
import glob

sns.set_context("talk", font_scale=1.0)

In [2]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'existing_relevant_combinations.csv'
model_list_kitti_top10 = ["liteflownet3_pseudoreg", "maskflownet", "flowformer++", "raft", "flow1d", "craft", "rpknet", "scopeflow", "gma", "ms_raft+"]
model_list_sintel_top10 = ["liteflownet3", "maskflownet", "flowformer++", "raft", "flow1d", "craft", "rpknet", "scopeflow", "gma", "ms_raft+"]
# Load the dataframe from CSV with comma as delimiter
df = pd.read_csv(file_path, delimiter=',')
model_names_mapping = {"raft": "RAFT", "gma": "GMA", "rpknet": "RPKNet", "ccmr": "CCMR", "craft": "CRAFT", "csflow":"CSFlow", "dicl": "DICL-Flow", "dip": "DIP", "fastflownet":"FastFlownet", "maskflownet":"MaskFlownet", "flow1d":"Flow1D", "flowformer": "FlowFormer", "flowformer++": "FlowFormer++", "gmflow": "GMFlow", "gmflownet": "GMFlowNet", "hd3": "HD3", "irr_pwc": "IRR", "liteflownet":"LiteFlowNet", "liteflownet2": "LiteFlowNet2", "liteflownet3": "LiteFlowNet3", "liteflownet3_pseudoreg": "LiteFlowNet3", "llaflow": "LLA-Flow", "ms_raft+": "MS-RAFT+", "rapidflow": "RAPIDFlow", "scopeflow": "ScopeFlow", "skflow": "SKFlow", "starflow": "STaRFlow", "videoflow_bof": "VideoFlow", "pwcnet": "PWCNet"}

marker_styles = {
    'raft': 'o',         # Circle
    'gma': 'X',          # Filled diamond
    'rpknet': 'v',       # Downward triangle
    'ccmr': 'h',         # Hexagon
    'craft': 's',        # Square
    'csflow': '^',       # Upward triangle
    'dicl': '<',         # Leftward triangle
    'dip': '>',          # Rightward triangle
    'fastflownet': 'o',  # Circle
    'maskflownet': 'v',  # Downward triangle
    'flow1d': '^',       # Upward triangle
    'flowformer': 'X',   # Filled diamond
    'flowformer++': 's', # Square
    'gmflow': '>',       # Rightward triangle
    'gmflownet': '<',    # Leftward triangle
    'hd3': 'h',          # Hexagon
    'irr_pwc': 's',      # Square
    'liteflownet': 'X',  # Filled diamond
    'liteflownet2': 'o', # Circle
    'liteflownet3': 'v', # Downward triangle
    'liteflownet3_pseudoreg': '^', # Upward triangle
    'llaflow': 'h',      # Hexagon
    'ms_raft+': '>',     # Rightward triangle
    'rapidflow': '<',     # Leftward triangle
    'scopeflow': 'X',    # Filled diamond
    'skflow': 'v',       # Plus (cross)
    'starflow': 's',     # Square
    'videoflow_bof': '^',# Upward triangle
    'pwcnet': 'o',       # Circle
}


line_styles = {
  'raft': '-',
  'gma': '--',
  'rpknet': '-.',
  'ccmr': ':',
  'craft': '-',
  'csflow': '--',
  'dicl': '-.',
  'dip': ':',
  'fastflownet': '-',
  'maskflownet': '--',
  'flow1d': '-.',
  'flowformer': ':',
  'flowformer++': '-',
  'gmflow': '--',
  'gmflownet': '-.',
  'hd3': ':',
  'irr_pwc': '-',
  'liteflownet': '--',
  'liteflownet2': '-.',
  'liteflownet3': ':',
  'liteflownet3_pseudoreg': '-',
  'llaflow': '--',
  'ms_raft+': '-.',
  'rapidflow': ':',
  'scopeflow': '-',
  'skflow': '--',
  'starflow': '-.',
  'videoflow_bof': ':',
  'pwcnet': '-'
}
# Use 'tab20' to get a larger set of distinct colors
color_palette = plt.get_cmap("tab10").colors
color_map = {}

for i in range(0, len(model_list_sintel_top10)):
    color_map[model_list_sintel_top10[i]] = color_palette[i]
    if model_list_sintel_top10[i] == "liteflownet3":
        color_map["liteflownet3_pseudoreg"]  = color_palette[i]

i = 0
for model in model_names_mapping.keys():
    if model not in color_map.keys():
        color_map[model] = color_palette[i]
        i = i + 1
    if i== 10:
      i = 0
  
#print(color_map)
# Create a color map by cycling through the larger 'tab20' palette
#color_map = {model: color_palette[i % len(color_palette)] for i, model in enumerate(marker_styles)}

#### BIM/PGD/CosPGD init flow as optimization target -- Kitti: Functions


In [5]:

# colors = plt.get_cmap("tab10").colors

# Create a distinct color palette for each model
# model_palette_sintel_top10 = {model: color for model, color in zip(model_list_sintel_top10, colors)}
# model_palette_kitti_top10 ={model: color for model, color in zip(model_list_kitti_top10, colors)}


def plot_optim(data, title, filename, optim_targets):
    num_optim_target = len(optim_targets)
    fig, axes = plt.subplots(1, num_optim_target, figsize=(5 * num_optim_target, 10))
    fig.suptitle(title)

    if num_optim_target == 1:
        axes = [axes]

    # Collect handles and labels for the legend
    handles, labels = [], []

    # Initialize y-limits
    y_min, y_max = float('inf'), float('-inf')

    for ax, optim_target in zip(axes, optim_targets):
        # Filter the data for the current optimization target
        plot_data = data.query(f"optim == '{optim_target}' or epsilon == 0")
        print(plot_data[['model', 'iterations_type', 'epes']].head())
        # Create a color palette using the color_map, matching models to their colors
        palette = {model: color_map.get(model, 'black') for model in plot_data['model'].unique()}

        # Create the line plot with consistent color mapping
        sns.lineplot(
            data=plot_data,
            x='iterations_type',
            y='epes',
            hue='model',
            palette=palette,  # Apply the color map to each model
            ax=ax,
            errorbar=None
        )

        # Update global y-limits
        current_y_min, current_y_max = ax.get_ylim()
        y_min = min(y_min, current_y_min)
        y_max = max(y_max, current_y_max)

        # Collect legend handles and labels from each plot
        h, l = ax.get_legend_handles_labels()
        handles.extend(h)
        # Map the raw model names to their pretty names
        labels.extend([model_names_mapping.get(label, label) for label in l])

        ax.set_title(f"Optimization Target: {optim_target.replace('ground_truth', 'Ground Truth').replace('initial_flow', 'Initial Flow')}")
        ax.set_xlabel('Num. of Attack Iterations')
        ax.set_ylabel('EPE w.r.t. Ground Truth')
        ax.grid(True)

    #Set the same y-limits for all axes
    for ax in axes:
        ax.set_ylim(y_min, y_max)

    # Remove individual legends from each subplot
    for ax in axes:
        ax.legend_.remove()

    # Create a unique legend for the entire figure using the mapped labels
    unique_handles_labels = dict(zip(labels, handles))
    fig.legend(unique_handles_labels.values(), unique_handles_labels.keys(), loc='upper center', bbox_to_anchor=(0.5, -0.001), ncol=4, frameon=False, title='Model')

    # Adjust layout to fit title and legend
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()




def plot_iteration(data, title, filename, epe_to_gt=True):

    fig, ax = plt.subplots(figsize=(12, 7))
    fig.suptitle(title)

    # Filter the data for the 'ground_truth' optimization target
    plot_data = data.query("optim == 'ground_truth' or epsilon == 0")

    # Create a color palette using the color_map, matching models to their colors
    palette = {model: color_map.get(model, 'black') for model in plot_data['model'].unique()}

    # Iterate over each model to plot with specific marker and line styles
    for model in plot_data['model'].unique():
        model_data = plot_data[plot_data['model'] == model]

        # Get the marker and line style for the current model, with a default fallback
        marker = marker_styles.get(model, 'o')  # Default marker is 'o'
        linestyle = line_styles.get(model, '-')  # Default line style is solid

        # Plot the continuous line for all data points (with markers but only for every 5th iteration)
        sns.lineplot(
            data=model_data,
            x='iterations_type',
            y='epes',
            ax=ax,
            linestyle=linestyle,
            color=palette[model],  # Use the color for the current model from the palette
            marker=marker,  # Use the marker style from global marker_styles
            markers=True,
            dashes=False,  # Keep solid lines
            markevery=5,  # Show marker only on every 5th point
            label=model_names_mapping.get(model, model),  # Map to pretty name if available
            errorbar=None
        )

    ax.set_xlabel('Num. of Attack Iterations')
    if epe_to_gt:
        ax.set_ylabel('EPE w.r.t. Ground Truth')
    else:
        ax.set_ylabel('EPE w.r.t. Target Flow')
    ax.grid(True)

    # Adjust legend
    plt.legend(title='Model', bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=4, frameon=False)

    # Adjust layout to give more space to the legend
    # plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust rect to move the bottom margin
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

def transform_data(df, model_list, query, wrt):
    filtered_df = df.query(query + " or (attack == 'none' and dataset == 'kitti-2015' and model in @model_list)")
    # Reshape the data
    melted_df = filtered_df.melt(id_vars=['model', 'epsilon', 'attack', 'optim'], value_vars=[f'epe_{wrt}_i{i}' for i in range(0, 21)], var_name='iterations_type', value_name='epes')
    melted_df['iterations_type'] = melted_df['iterations_type'].str.extract('(\d+)').astype(int)
    return melted_df



#### BIM/PGD/CosPGD init flow as optimization target -- Kitti

In [ ]:
### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_optim(data_untargeted_inf, r'KITTI-2015 - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_kitti_init_flow.png', ["initial_flow", "ground_truth"])


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_optim(data_untargeted_inf, r'KITTI-2015 - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_kitti_init_flow.png', ["initial_flow", "ground_truth"])


#### BIM/PGD/CosPGD top 10 models -- Kitti Inf


In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_untargeted_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'KITTI-2015 - BIM - Untargeted - $L_{\infty}$', 'graphs/BIM_kitti_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_negative_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_kitti_top_10_negative_inf_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_zero_inf = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_kitti_top_10_zero_inf_gt.png', False)

# Plot for targeted attack (Negative Target) with respect to the target
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_negative_inf = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_kitti_top_10_negative_inf_target.png', False)

# Plot for targeted attack (Zero Target) with respect to the ground truth
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_zero_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_kitti_top_10_zero_inf_target.png')



### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_untargeted_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'KITTI-2015 - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_kitti_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_negative_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_kitti_top_10_negative_inf_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_zero_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_kitti_top_10_zero_inf_gt.png')

# Plot for targeted attack (Negative Target) with respect to the target
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_negative_inf = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_kitti_top_10_negative_inf_target.png', False)

# Plot for targeted attack (Zero Target) with respect to the target
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_zero_inf = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_kitti_top_10_zero_inf_target.png', False)


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_untargeted_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'KITTI-2015 - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_kitti_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_negative_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_kitti_top_10_negative_inf_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_zero_inf = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_kitti_top_10_zero_inf_gt.png')

# Plot for targeted attack (Negative Target) with respect to the target
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_negative_inf = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_kitti_top_10_negative_inf_target.png', False)

# Plot for targeted attack (Zero Target) with respect to the target
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_kitti_top10"
data_targeted_zero_inf = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_kitti_top_10_zero_inf_target.png', False)



#### BIM/PGD/CosPGD all models -- Kitti Inf


In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'KITTI-2015 - BIM - Untargeted - $L_{\infty}$', 'graphs/BIM_kitti_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_kitti_negative_inf_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_kitti_zero_inf_gt.png', False)

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_kitti_negative_inf_target.png', False)

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_kitti_zero_inf_target.png')


### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'KITTI-2015 - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_kitti_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_kitti_negative_inf_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_kitti_zero_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_kitti_negative_inf_target.png', False)

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_kitti_zero_inf_target.png', False)

### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'KITTI-2015 - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_kitti_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_kitti_negative_inf_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_kitti_zero_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'KITTI-2015 - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_kitti_negative_inf_target.png', False)

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'KITTI-2015 - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_kitti_zero_inf_target.png', False)


### BIM/PG/CosPGD Top 10 Models -- Kitti Two

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'two' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_untargeted_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'KITTI-2015 - BIM - Untargeted - $L_{2}$', 'graphs/BIM_kitti_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_negative_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - BIM - Negative Target - $L_{2}$', 'graphs/BIM_kitti_top_10_negative_two_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_zero_two = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - BIM - Zero Target - $L_{2}$', 'graphs/BIM_kitti_top_10_zero_two_gt.png', False)

# Plot for targeted attack (Negative Target) with respect to the target
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_negative_two = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - BIM - Negative Target - $L_{2}$', 'graphs/BIM_kitti_top_10_negative_two_target.png', False)

# Plot for targeted attack (Zero Target) with respect to the ground truth
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_zero_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - BIM - Zero Target - $L_{2}$', 'graphs/BIM_kitti_top_10_zero_two_target.png')


### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'two' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_untargeted_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'KITTI-2015 - PGD - Untargeted - $L_{2}$', 'graphs/PGD_kitti_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_negative_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - PGD - Negative Target - $L_{2}$', 'graphs/PGD_kitti_top_10_negative_two_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_zero_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - PGD - Zero Target - $L_{2}$', 'graphs/PGD_kitti_top_10_zero_two_gt.png')

# Plot for targeted attack (Negative Target) with respect to the target
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_negative_two = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - PGD - Negative Target - $L_{2}$', 'graphs/PGD_kitti_top_10_negative_two_target.png', False)

# Plot for targeted attack (Zero Target) with respect to the target
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_zero_two = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - PGD - Zero Target - $L_{2}$', 'graphs/PGD_kitti_top_10_zero_two_target.png', False)


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'two' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_untargeted_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'KITTI-2015 - CosPGD - Untargeted - $L_{2}$', 'graphs/CosPGD_kitti_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_negative_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_kitti_top_10_negative_two_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_zero_two = transform_data(df, model_list_kitti_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_kitti_top_10_zero_two_gt.png')

# Plot for targeted attack (Negative Target) with respect to the target
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_negative_two = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_kitti_top_10_negative_two_target.png', False)

# Plot for targeted attack (Zero Target) with respect to the target
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_kitti_top10"
data_targeted_zero_two = transform_data(df, model_list_kitti_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_kitti_top_10_zero_two_target.png', False)



#### BIM/PGD/CosPGD all models -- Kitti Two


In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'two' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'KITTI-2015 - BIM - Untargeted - $L_{2}$', 'graphs/BIM_kitti_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - BIM - Negative Target - $L_{2}$', 'graphs/BIM_kitti_negative_two_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - BIM - Zero Target - $L_{2}$', 'graphs/BIM_kitti_zero_two_gt.png', False)

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - BIM - Negative Target - $L_{2}$', 'graphs/BIM_kitti_negative_two_target.png', False)

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - BIM - Zero Target - $L_{2}$', 'graphs/BIM_kitti_zero_two_target.png')

### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'two' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'KITTI-2015 - PGD - Untargeted - $L_{2}$', 'graphs/PGD_kitti_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - PGD - Negative Target - $L_{2}$', 'graphs/PGD_kitti_negative_two_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - PGD - Zero Target - $L_{2}$', 'graphs/PGD_kitti_zero_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - PGD - Negative Target - $L_{2}$', 'graphs/PGD_kitti_negative_two_target.png', False)

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - PGD - Zero Target - $L_{2}$', 'graphs/PGD_kitti_zero_two_target.png', False)

### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'two' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'KITTI-2015 - CosPGD - Untargeted - $L_{2}$', 'graphs/CosPGD_kitti_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_kitti_negative_two_gt.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_kitti_zero_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'KITTI-2015 - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_kitti_negative_two_target.png', False)

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'KITTI-2015 - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_kitti_zero_two_target.png', False)


#### BIM/PGD/CosPGD init flow as optimization target -- Sintel-clean

In [ ]:
def transform_data(df, model_list, query, wrt):
    filtered_df = df.query(query + " or (attack == 'none' and dataset == 'sintel-clean' and model in @model_list)")
    # Reshape the data
    melted_df = filtered_df.melt(id_vars=['model', 'epsilon', 'attack', 'optim'], value_vars=[f'epe_{wrt}_i{i}' for i in range(0, 21)], var_name='iterations_type', value_name='epes')
    melted_df['iterations_type'] = melted_df['iterations_type'].str.extract('(\d+)').astype(int)
    return melted_df


### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_optim(data_untargeted_inf, r'Sintel-Clean - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_sintel_clean_init_flow.png', ["initial_flow", "ground_truth"])


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_optim(data_untargeted_inf, r'Sintel-Clean - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_init_flow.png', ["initial_flow", "ground_truth"])


### BIM/PGD/CosPGD Top10 models -- SIntel-clean INf

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Clean - BIM - Untargeted - $L_{\infty}$', 'graphs/BIM_sintel_clean_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_sintel_clean_top_10_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_sintel_clean_top_10_zero_target_inf.png', False)

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_sintel_clean_top_10_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_sintel_clean_top_10_zero_target_inf_wrt_target.png', False)



### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Clean - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_sintel_clean_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_top_10_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_top_10_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_top_10_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_top_10_zero_target_inf_wrt_target.png', False)

### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Clean - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_top_10_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_top_10_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_top_10_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_top_10_zero_target_inf_wrt_target.png', False)




#### BIM/PGD/CosPGD all models -- Sintel-clean Inf

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Clean - BIM - Untargeted - $L_{\infty}$', 'graphs/BIM_sintel_clean_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_sintel_clean_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_sintel_clean_zero_target_inf_wrt_target.png', False)



### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Clean - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_sintel_clean_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_clean_zero_target_inf_wrt_target.png', False)


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Clean - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Clean - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Clean - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_sintel_clean_zero_target_inf_wrt_target.png', False)



#### BIM/PGD/CosPGD top10 models -- Sintel-clean Two

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'two' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_untargeted_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Clean - BIM - Untargeted - $L_{2}$', 'graphs/BIM_sintel_clean_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - BIM - Negative Target - $L_{2}$', 'graphs/BIM_sintel_clean_top_10_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - BIM - Zero Target - $L_{2}$', 'graphs/BIM_sintel_clean_top_10_zero_target_two.png', False)

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - BIM - Negative Target - $L_{2}$', 'graphs/BIM_sintel_clean_top_10_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - BIM - Zero Target - $L_{2}$', 'graphs/BIM_sintel_clean_top_10_zero_target_two_wrt_target.png', False)


### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'two' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_untargeted_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Clean - PGD - Untargeted - $L_{2}$', 'graphs/PGD_sintel_clean_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_clean_top_10_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_clean_top_10_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_clean_top_10_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_clean_top_10_zero_target_two_wrt_target.png', False)


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'two' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_untargeted_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Clean - CosPGD - Untargeted - $L_{2}$', 'graphs/CosPGD_sintel_clean_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_sintel_clean_top_10_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_sintel_clean_top_10_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_sintel_clean_top_10_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_sintel_clean_top_10_zero_target_two_wrt_target.png', False)



#### BIM/PGD/CosPGD all models -- Sintel-clean Two

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'two' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Clean - BIM - Untargeted - $L_{2}$', 'graphs/BIM_sintel_clean_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_clean_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_clean_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - BIM - Negative Target - $L_{2}$', 'graphs/BIM_sintel_clean_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - BIM - Zero Target - $L_{2}$', 'graphs/BIM_sintel_clean_zero_target_two_wrt_target.png', False)


### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'two' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Clean - PGD - Untargeted - $L_{2}$', 'graphs/PGD_sintel_clean_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_clean_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_clean_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_clean_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_clean_zero_target_two_wrt_target.png', False)


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'two' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Clean - CosPGD - Untargeted - $L_{2}$', 'graphs/CosPGD_sintel_clean_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_sintel_clean_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_sintel_clean_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Clean - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_sintel_clean_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Clean - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_sintel_clean_zero_target_two_wrt_target.png', False)


#### BIM/PGD/CosPGD init flow as optimization target -- Sintel-final

In [ ]:
def transform_data(df, model_list, query, wrt):
    filtered_df = df.query(query + " or (attack == 'none' and dataset == 'sintel-final' and model in @model_list)")
    # Reshape the data
    melted_df = filtered_df.melt(id_vars=['model', 'epsilon', 'attack', 'optim'], value_vars=[f'epe_{wrt}_i{i}' for i in range(0, 21)], var_name='iterations_type', value_name='epes')
    melted_df['iterations_type'] = melted_df['iterations_type'].str.extract('(\d+)').astype(int)
    return melted_df

### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_optim(data_untargeted_inf, r'Sintel-Final - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_sintel_final_init_flow.png', ["initial_flow", "ground_truth"])


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_optim(data_untargeted_inf, r'Sintel-Final - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_sintel_final_init_flow.png', ["initial_flow", "ground_truth"])


#### BIM/PGD/CosPGD top10 models -- Sintel-final Inf

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'inf' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Final - BIM - Untargeted - $L_{\infty}$', 'graphs/BIM_sintel_final_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_sintel_final_top_10_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_sintel_final_top_10_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_sintel_final_top_10_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_sintel_final_top_10_zero_target_inf_wrt_target.png', False)



### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Final - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_sintel_final_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_final_top_10_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_final_top_10_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_final_top_10_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_final_top_10_zero_target_inf_wrt_target.png', False)


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_untargeted_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Final - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_sintel_final_top_10_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_sintel_final_top_10_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_sintel_final_top_10_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_negative_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_sintel_final_top_10_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list_sintel_top10"
data_targeted_zero_inf = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_sintel_final_top_10_zero_target_inf_wrt_target.png', False)



#### BIM/PGD/CosPGD all models -- Sintel-final Inf

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'inf' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Final - BIM - Untargeted - $L_{\infty}$', 'graphs/BIM_sintel_final_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_sintel_final_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_sintel_final_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - BIM - Negative Target - $L_{\infty}$', 'graphs/BIM_sintel_final_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'bim' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - BIM - Zero Target - $L_{\infty}$', 'graphs/BIM_sintel_final_zero_target_inf_wrt_target.png', False)



### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'inf' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Final - PGD - Untargeted - $L_{\infty}$', 'graphs/PGD_sintel_final_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_final_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_final_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - PGD - Negative Target - $L_{\infty}$', 'graphs/PGD_sintel_final_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - PGD - Zero Target - $L_{\infty}$', 'graphs/PGD_sintel_final_zero_target_inf_wrt_target.png', False)



### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'inf' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_untargeted_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_inf, r'Sintel-Final - CosPGD - Untargeted - $L_{\infty}$', 'graphs/CosPGD_sintel_final_untargeted_inf_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_sintel_final_negative_target_inf.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_sintel_final_zero_target_inf.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_negative_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_inf, r'Sintel-Final - CosPGD - Negative Target - $L_{\infty}$', 'graphs/CosPGD_sintel_final_negative_target_inf_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.0314 and model in @model_list"
data_targeted_zero_inf = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_inf, r'Sintel-Final - CosPGD - Zero Target - $L_{\infty}$', 'graphs/CosPGD_sintel_final_zero_target_inf_wrt_target.png', False)



#### BIM/PGD/CosPGD top10 models -- Sintel-final Two

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'two' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_untargeted_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Final - BIM - Untargeted - $L_{2}$', 'graphs/BIM_sintel_final_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - BIM - Negative Target - $L_{2}$', 'graphs/BIM_sintel_final_top_10_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - BIM - Zero Target - $L_{2}$', 'graphs/BIM_sintel_final_top_10_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - BIM - Negative Target - $L_{2}$', 'graphs/BIM_sintel_final_top_10_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - BIM - Zero Target - $L_{2}$', 'graphs/BIM_sintel_final_top_10_zero_target_two_wrt_target.png', False)


### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'two' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_untargeted_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Final - PGD - Untargeted - $L_{2}$', 'graphs/PGD_sintel_final_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_final_top_10_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_final_top_10_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_final_top_10_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_final_top_10_zero_target_two_wrt_target.png', False)

### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'two' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_untargeted_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Final - CosPGD - Untargeted - $L_{2}$', 'graphs/CosPGD_sintel_final_top_10_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_sintel_final_top_10_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_sintel_final_top_10_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_negative_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_sintel_final_top_10_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list_sintel_top10"
data_targeted_zero_two = transform_data(df, model_list_sintel_top10, query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_sintel_final_top_10_zero_target_two_wrt_target.png', False)


#### BIM/PGD/CosPGD all models -- Sintel-final Two

In [ ]:
### BIM ###

# Plot for untargeted attack
query = "attack == 'bim' and norm == 'two' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Final - BIM - Untargeted - $L_{2}$', 'graphs/BIM_sintel_final_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - BIM - Negative Target - $L_{2}$', 'graphs/BIM_sintel_final_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - BIM - Zero Target - $L_{2}$', 'graphs/BIM_sintel_final_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - BIM - Negative Target - $L_{2}$', 'graphs/BIM_sintel_final_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'bim' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - BIM - Zero Target - $L_{2}$', 'graphs/BIM_sintel_final_zero_target_two_wrt_target.png', False)


### PGD ###

# Plot for untargeted attack
query = "attack == 'pgd' and norm == 'two' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Final - PGD - Untargeted - $L_{2}$', 'graphs/PGD_sintel_final_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_final_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_final_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - PGD - Negative Target - $L_{2}$', 'graphs/PGD_sintel_final_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'pgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - PGD - Zero Target - $L_{2}$', 'graphs/PGD_sintel_final_zero_target_two_wrt_target.png', False)


### CosPGD ###

# Plot for untargeted attack
query = "attack == 'cospgd' and norm == 'two' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_untargeted_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_untargeted_two, r'Sintel-Final - CosPGD - Untargeted - $L_{2}$', 'graphs/CosPGD_sintel_final_untargeted_two_gt.png')

# Plot for targeted attack (Negative Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_sintel_final_negative_target_two.png')

# Plot for targeted attack (Zero Target)
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="gt")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_sintel_final_zero_target_two.png')

# Plot for targeted attack (Negative Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_negative_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_negative_two, r'Sintel-Final - CosPGD - Negative Target - $L_{2}$', 'graphs/CosPGD_sintel_final_negative_target_two_wrt_target.png', False)

# Plot for targeted attack (Zero Target) with respect to 'target'
query = "attack == 'cospgd' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') and epsilon == 0.251 and model in @model_list"
data_targeted_zero_two = transform_data(df, model_names_mapping.keys(), query, wrt="target")
plot_iteration(data_targeted_zero_two, r'Sintel-Final - CosPGD - Zero Target - $L_{2}$', 'graphs/CosPGD_sintel_final_zero_target_two_wrt_target.png', False)


### FGSM Methods



In [30]:
specific_models = ['raft', 'craft', 'gma', 'ms_raft+', 'flowformer++', 'rpknet', 'flow1d', 'liteflownet3', 'scopeflow', 'maskflownet']
blacklist = ["hd3"]

def plot_fgsm(data, title, filename, epe_to_ground_truth=True):
    targeted = data['targeted'].all()
    # Create the plot with additional styles for lines and markers
    plt.figure(figsize=(10, 8))

    # Iterate over models and plot each individually with custom markers, line styles, and colors
    for model in data['model'].unique():
        model_data = data[data['model'] == model]
        
        # Use the model names mapping for pretty names and apply the marker, line styles, and colors
        pretty_model_name = model_names_mapping.get(model, model)
        marker = marker_styles.get(model, 'o')  # Default to 'o' if no marker specified
        linestyle = line_styles.get(model, '-')  # Default to solid line if no linestyle specified
        color = color_map.get(model)  # Get color from the color map

        if epe_to_ground_truth and targeted:
            sns.lineplot(
                data=model_data,
                x='epsilon',
                y='epe_ground_truth',
                label=pretty_model_name,
                marker=marker,
                linestyle=linestyle,
                color=color,
                errorbar=None,  # Apply custom color for each model
            )
        else:
            sns.lineplot(
                data=model_data,
                x='epsilon',
                y='epe',
                label=pretty_model_name,
                marker=marker,
                linestyle=linestyle,
                color=color,
                errorbar=None,  # Apply custom color for each model
            )

    # Customize the plot
    plt.xlabel('Epsilon')
    if epe_to_ground_truth:
        plt.ylabel('EPE w.r.t. Ground Truth')
    else:
        plt.ylabel('EPE w.r.t. Target Flow')
    
    plt.grid(True)
    plt.title(title)

    # Remove annotations (previously added with plt.text)

    # Adjust legend
    plt.legend(title='Model', bbox_to_anchor=(0.5, -0.15), loc='upper center', ncol=4, frameon=False)

    # Show and save the plot
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()



def plot_fgsm_scatter(data, title, filename, epe_to_ground_truth=True):
    targeted = data['targeted'].all()
    # Increase figure size
    plt.figure(figsize=(12, 7))  # Adjust the figure size to increase the plot area

    # Replace model names with pretty names
    data['pretty_model_name'] = data['model'].apply(lambda x: model_names_mapping.get(x, x))

    # Create a numerical mapping for the model names
    model_mapping = {model: idx for idx, model in enumerate(data['pretty_model_name'].unique())}
    data['model_position'] = data['pretty_model_name'].map(model_mapping)

    # Apply marker styles based on the model
    data['marker'] = data['model'].apply(lambda x: marker_styles.get(x, 'o'))  # Default marker is 'o'

    # Create the scatter plot manually to apply custom markers and colors
    for model in data['model'].unique():
        model_data = data[data['model'] == model]
        pretty_model_name = model_names_mapping.get(model, model)
        marker = marker_styles.get(model, 'o')  # Get marker for the model
        color = color_map.get(model)  # Get color from the color map

        if epe_to_ground_truth and targeted:
            sns.scatterplot(
                data=model_data, 
                x='model_position', 
                y='epe_ground_truth', 
                label=pretty_model_name, 
                marker=marker,           # Apply custom marker for each model
                color=color,             # Apply custom color for each model
                s=100                    # Marker size
            )
        else:
            sns.scatterplot(
                data=model_data, 
                x='model_position', 
                y='epe', 
                label=pretty_model_name, 
                marker=marker,           # Apply custom marker for each model
                color=color,             # Apply custom color for each model
                s=100                    # Marker size
            )

    # Remove the x-axis labels (bottom model names)
    plt.gca().axes.get_xaxis().set_visible(False)

    # Customize the plot
    if epe_to_ground_truth:
        plt.ylabel('EPE w.r.t. Ground Truth')
    else:
        plt.ylabel('EPE w.r.t. Target Flow')
    plt.grid(True, axis='y')
    plt.title(title)

    # Scale down the legend font size
    plt.legend(
        title='Model', 
        bbox_to_anchor=(0.5, -0.15), 
        loc='upper center', 
        ncol=4, 
        frameon=False, 
        fontsize='xx-small',      # Decreased font size
        handletextpad=0.5,        # Reduce spacing between the legend icon and text
        labelspacing=0.4          # Reduce vertical space between legend entries
    )

    # Save the plot with reduced padding
    plt.tight_layout(pad=2)
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()




### FGSM L-infinity Kitti

In [ ]:
# Plot for untargeted attack
data_untargeted = df.query("model not in @blacklist and attack == 'fgsm' and norm == 'inf' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') or (attack == 'none' and (checkpoint == 'kitti' or checkpoint == 'mixed'))")
plot_fgsm(data_untargeted, r'KITTI-2015 - FGSM - Untargeted - $L_{\infty}$', 'graphs/FGSM_kitti_untargeted_inf_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_zero_inf.png')
plot_fgsm(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_zero_inf_gt.png')

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_negative_inf.png')
plot_fgsm(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_negative_inf_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_zero_inf.png')
plot_fgsm(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_zero_inf_target.png', False)

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_negative_inf.png')
plot_fgsm(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_negative_inf_target.png', False)

# Plot for untargeted attack with specific models
data_specific_models_untargeted = data_untargeted[data_untargeted['model'].isin(specific_models)]
plot_fgsm(data_specific_models_untargeted, r'KITTI-2015 - FGSM - Untargeted - $L_{\infty}$', 'graphs/FGSM_kitti_top10_untargeted_inf_gt.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGS_kitti_top10_zero_inf.png')
plot_fgsm(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_zero_inf_gt.png')

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_negative_inf.png')
plot_fgsm(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_negative_inf_gt.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGS_kitti_top10_zero_inf.png')
plot_fgsm(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_zero_inf_target.png', False)

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_negative_inf.png')
plot_fgsm(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_negative_inf_target.png', False)

### FGSM L-2 Kitti

In [ ]:
# Plot for untargeted attack
data_untargeted = df.query("model not in @blacklist and attack == 'fgsm' and norm == 'two' and targeted == False and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed') or (attack == 'none' and (checkpoint == 'kitti' or checkpoint == 'mixed'))")
plot_fgsm(data_untargeted, r'KITTI-2015 - FGSM - Untargeted - $L_{2}$', 'graphs/FGSM_kitti_untargeted_two_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed')")
plot_fgsm(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_kitti_zero_two_gt.png')

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed')")
plot_fgsm(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_kitti_negative_two_gt.png')

# Plot for targeted attack with target zero (with respect to 'target')
data_target_zero = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed')")
plot_fgsm(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_kitti_zero_two_target.png', False)

# Plot for targeted attack with negative target (with respect to 'target')
data_target_negative = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'kitti-2015' and (checkpoint == 'kitti' or checkpoint == 'mixed')")
plot_fgsm(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_kitti_negative_two_target.png', False)

# Plot for untargeted attack with specific models
data_specific_models_untargeted = data_untargeted[data_untargeted['model'].isin(specific_models)]
plot_fgsm(data_specific_models_untargeted, r'KITTI-2015 - FGSM - Untargeted - $L_{2}$', 'graphs/FGSM_kitti_top10_untargeted_two_gt.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_kitti_top10_zero_two_gt.png')

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_kitti_top10_negative_two_gt.png')

# Plot for targeted attack with target zero and specific models (with respect to 'target')
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_kitti_top10_zero_two_target.png', False)

# Plot for targeted attack with negative target and specific models (with respect to 'target')
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_kitti_top10_negative_two_target.png', False)


### FGSM L-infinity Sintel-Clean

In [ ]:
# Plot for untargeted attack
data_untargeted = df.query("attack == 'fgsm' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') or (attack == 'none' and (checkpoint == 'sintel' or checkpoint == 'mixed'))")
plot_fgsm(data_untargeted, r'Sintel-Clean - FGSM - Untargeted - $L_{\infty}$', 'graphs/FGSM_sintel_clean_untargeted_inf_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_zero_inf.png')
plot_fgsm(data_target_zero, r'Sintel-Clean - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_sintel_clean_inf_gt.png')

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_negative_inf.png')
plot_fgsm(data_target_negative, r'Sintel-Clean - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_sintel_clean_negative_inf_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_zero_inf.png')
plot_fgsm(data_target_zero, r'Sintel-Clean - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_sintel_clean_inf_target.png', False)

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_negative_inf.png')
plot_fgsm(data_target_negative, r'Sintel-Clean - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_sintel_clean_negative_inf_target.png', False)

# Plot for untargeted attack with specific models
data_specific_models_untargeted = data_untargeted[data_untargeted['model'].isin(specific_models)]
plot_fgsm(data_specific_models_untargeted, r'Sintel-Clean - FGSM - Untargeted - $L_{\infty}$', 'graphs/FGSM_sintel_clean_top10_untargeted_inf.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGS_kitti_top10_zero_inf.png')
plot_fgsm(data_specific_models_target_zero, r'Sintel-Clean - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_sintel_clean_top10_zero_inf_gt.png')

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_negative_inf.png')
plot_fgsm(data_specific_models_target_negative, r'Sintel-Clean - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_sintel_clean_top10_negative_inf_gt.png')

# Plot for untargeted attack with specific models
data_specific_models_untargeted = data_untargeted[data_untargeted['model'].isin(specific_models)]
plot_fgsm(data_specific_models_untargeted, r'Sintel-Clean - FGSM - Untargeted - $L_{\infty}$', 'graphs/FGSM_sintel_clean_top10_untargeted_inf_gt.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGS_kitti_top10_zero_inf.png')
plot_fgsm(data_specific_models_target_zero, r'Sintel-Clean - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_sintel_clean_top10_zero_inf_target.png', False)

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_negative_inf.png')
plot_fgsm(data_specific_models_target_negative, r'Sintel-Clean - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_sintel_clean_top10_negative_inf_target.png', False)

### FGSM L-2 Sintel-Clean

In [ ]:
# Plot for untargeted attack
data_untargeted = df.query("attack == 'fgsm' and norm == 'two' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') or (attack == 'none' and (checkpoint == 'sintel' or checkpoint == 'mixed'))")
plot_fgsm(data_untargeted, r'Sintel-Clean - FGSM - Untargeted - $L_{2}$', 'graphs/FGSM_sintel_clean_untargeted_two_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
plot_fgsm(data_target_zero, r'Sintel-Clean - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_sintel_clean_zero_two_gt.png')

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
plot_fgsm(data_target_negative, r'Sintel-Clean - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_sintel_clean_negative_two_gt.png')

# Plot for targeted attack with target zero (with respect to 'target')
data_target_zero = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
plot_fgsm(data_target_zero, r'Sintel-Clean - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_sintel_clean_zero_two_target.png', False)

# Plot for targeted attack with negative target (with respect to 'target')
data_target_negative = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
plot_fgsm(data_target_negative, r'Sintel-Clean - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_sintel_clean_negative_two_target.png', False)

# Plot for untargeted attack with specific models
data_specific_models_untargeted = data_untargeted[data_untargeted['model'].isin(specific_models)]
plot_fgsm(data_specific_models_untargeted, r'Sintel-Clean - FGSM - Untargeted - $L_{2}$', 'graphs/FGSM_sintel_clean_top10_untargeted_two_gt.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_zero, r'Sintel-Clean - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_sintel_clean_top10_zero_two_gt.png')

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_negative, r'Sintel-Clean - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_sintel_clean_top10_negative_two_gt.png')

# Plot for targeted attack with target zero and specific models (with respect to 'target')
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_zero, r'Sintel-Clean - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_sintel_clean_top10_zero_two_target.png', False)

# Plot for targeted attack with negative target and specific models (with respect to 'target')
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_negative, r'Sintel-Clean - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_sintel_clean_top10_negative_two_target.png', False)


### FGSM L-infinity Sintel-Final

In [ ]:
# Plot for untargeted attack
data_untargeted = df.query("attack == 'fgsm' and norm == 'inf' and targeted == False and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed') or (attack == 'none' and (checkpoint == 'sintel' or checkpoint == 'mixed'))")
plot_fgsm(data_untargeted, r'Sintel-Final - FGSM - Untargeted - $L_{\infty}$', 'graphs/FGSM_sintel_final_untargeted_inf_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_zero_inf.png')
plot_fgsm(data_target_zero, r'Sintel-Final - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_sintel_final_inf_gt.png')

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_negative_inf.png')
plot_fgsm(data_target_negative, r'Sintel-Final - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_sintel_final_negative_inf_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'zero' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_kitti_zero_inf.png')
plot_fgsm(data_target_zero, r'Sintel-Final - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_sintel_final_inf_target.png', False)

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'inf' and targeted == True and target == 'negative' and dataset == 'sintel-clean' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
#plot_fgsm_scatter(data_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_negative_inf.png')
plot_fgsm(data_target_negative, r'Sintel-Final - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_sintel_final_negative_inf_target.png', False)



# Plot for untargeted attack with specific models
data_specific_models_untargeted = data_untargeted[data_untargeted['model'].isin(specific_models)]
plot_fgsm(data_specific_models_untargeted, r'Sintel-Final - FGSM - Untargeted - $L_{\infty}$', 'graphs/FGSM_sintel_final_top10_untargeted_inf_gt.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGS_kitti_top10_zero_inf.png')
plot_fgsm(data_specific_models_target_zero, r'Sintel-Final - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_sintel_final_top10_zero_inf_gt.png')

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_negative_inf.png')
plot_fgsm(data_specific_models_target_negative, r'Sintel-Final - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_sintel_final_top10_negative_inf_gt.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_zero, r'KITTI-2015 - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGS_kitti_top10_zero_inf.png')
plot_fgsm(data_specific_models_target_zero, r'Sintel-Final - FGSM - Zero-Target - $L_{\infty}$', 'graphs/FGSM_sintel_final_top10_zero_inf_target.png', False)

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
#plot_fgsm_scatter(data_specific_models_target_negative, r'KITTI-2015 - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_kitti_top10_negative_inf.png')
plot_fgsm(data_specific_models_target_negative, r'Sintel-Final - FGSM - Negative-Target - $L_{\infty}$', 'graphs/FGSM_sintel_final_top10_negative_inf_target.png', False)

### FGSM L-2 Sintel-Final

In [ ]:
# Plot for untargeted attack
data_untargeted = df.query("attack == 'fgsm' and norm == 'two' and targeted == False and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed') or (attack == 'none' and (checkpoint == 'sintel' or checkpoint == 'mixed'))")
plot_fgsm(data_untargeted, r'Sintel-Final - FGSM - Untargeted - $L_{2}$', 'graphs/FGSM_sintel_final_untargeted_two_gt.png')

# Plot for targeted attack with target zero
data_target_zero = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
plot_fgsm(data_target_zero, r'Sintel-Final - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_sintel_final_zero_two_gt.png')

# Plot for targeted attack with negative target
data_target_negative = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
plot_fgsm(data_target_negative, r'Sintel-Final - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_sintel_final_negative_two_gt.png')

# Plot for targeted attack with target zero (with respect to 'target')
data_target_zero = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'zero' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
plot_fgsm(data_target_zero, r'Sintel-Final - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_sintel_final_zero_two_target.png', False)

# Plot for targeted attack with negative target (with respect to 'target')
data_target_negative = df.query("attack == 'fgsm' and norm == 'two' and targeted == True and target == 'negative' and dataset == 'sintel-final' and (checkpoint == 'sintel' or checkpoint == 'mixed')")
plot_fgsm(data_target_negative, r'Sintel-Final - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_sintel_final_negative_two_target.png', False)


# Plot for untargeted attack with specific models
data_specific_models_untargeted = data_untargeted[data_untargeted['model'].isin(specific_models)]
plot_fgsm(data_specific_models_untargeted, r'Sintel-Final - FGSM - Untargeted - $L_{2}$', 'graphs/FGSM_sintel_final_top10_untargeted_two_gt.png')

# Plot for targeted attack with target zero and specific models
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_zero, r'Sintel-Final - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_sintel_final_top10_zero_two_gt.png')

# Plot for targeted attack with negative target and specific models
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_negative, r'Sintel-Final - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_sintel_final_top10_negative_two_gt.png')

# Plot for targeted attack with target zero and specific models (with respect to 'target')
data_specific_models_target_zero = data_target_zero[data_target_zero['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_zero, r'Sintel-Final - FGSM - Zero-Target - $L_{2}$', 'graphs/FGSM_sintel_final_top10_zero_two_target.png', False)

# Plot for targeted attack with negative target and specific models (with respect to 'target')
data_specific_models_target_negative = data_target_negative[data_target_negative['model'].isin(specific_models)]
plot_fgsm(data_specific_models_target_negative, r'Sintel-Final - FGSM - Negative-Target - $L_{2}$', 'graphs/FGSM_sintel_final_top10_negative_two_target.png', False)
